In [3]:
import sys
print(sys.executable)

/opt/miniconda/envs/ksa/bin/python


In [4]:
import pandas as pd
import numpy as np
import traceback

from esa_snappy import ProductIO
from esa_snappy import GeoPos
from esa_snappy import PixelPos

from glob import glob
from tqdm import tqdm
import os
import pickle

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
import pandas as pd
from tqdm import tqdm

# Suppress specific warnings
warnings.filterwarnings("ignore", category=FutureWarning, module='pandas', lineno=11)
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)

INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.
Currently installed 10.0.0, available is 11.0.0.
Please visit http://step.esa.int



In [5]:
data = 'imputed'
idprov = '95'

In [6]:
## Get ID and Year 
years = range(2021, 2024)
numbers = range(1, 31)
year_id_ = []
for year in years:
    for number in numbers:
        year_id_.append(f"{year}_{str(number).zfill(2)}")
year_id_[:5]

vh_list = [f"VH_{i}" for i in range(30, -1, -1)]
print(vh_list)

# vv_list = [f"VV_{i}" for i in range(30, -1, -1)]
# print(vv_list)


['VH_30', 'VH_29', 'VH_28', 'VH_27', 'VH_26', 'VH_25', 'VH_24', 'VH_23', 'VH_22', 'VH_21', 'VH_20', 'VH_19', 'VH_18', 'VH_17', 'VH_16', 'VH_15', 'VH_14', 'VH_13', 'VH_12', 'VH_11', 'VH_10', 'VH_9', 'VH_8', 'VH_7', 'VH_6', 'VH_5', 'VH_4', 'VH_3', 'VH_2', 'VH_1', 'VH_0']


In [7]:
## Prep DF
df_all_wide = pd.DataFrame(columns=["idpoint"]+year_id_)
# df_all_wide

df_bridging_citra = pd.read_excel("/data/ksa/03_Sampling/bridging.xlsx", dtype='object', sheet_name="periode_to_date")
# df_bridging_citra.head(2)

df_bridging_ksa = pd.read_excel("/data/ksa/03_Sampling/bridging.xlsx", dtype='object')
# df_bridging_ksa.head(2)

In [8]:
df_bridging_ksa

,is_kabisat,obs_in_a_year,id_per_image,periode_start,periode_end
0,0,1,3,0125,0205
1,0,2,5,0218,0301
2,0,3,8,0326,0406
3,0,4,10,0419,0430
4,0,5,13,0525,0605
5,0,6,15,0618,0629
6,0,7,18,0724,0804
7,0,8,20,0817,0828
8,0,9,23,0922,1003
9,0,10,25,1016,1027


In [9]:
def get_df_values(data, idprov, mgrs):
    if(data == 'not imputed'):
        with open('/data/ksa/03_Sampling/data/'+idprov+'/sampling_'+mgrs+'.pkl', 'rb') as f:
            df_values = pickle.load(f)    
        df_values["VH"] = df_values.Sigma0_VH_db
        df_values.drop(["Sigma0_VH_db"],axis=True, inplace=True)
    elif(data == 'imputed'):
        with open('/data/ksa/04_Data_Preprocessing/'+idprov+'/01_imputation/'+mgrs+'_imputed_data.pkl', 'rb') as f:
            df_values = pickle.load(f) 
        df_values["VH"] = df_values.Sigma0_VH_db_imputation
        df_values.drop(["Sigma0_VH_db_imputation"],axis=True, inplace=True)
    return df_values

def reformat_to_wide(df_values, band, df_bridging_citra, df_all_wide):
    
    df_values["periode_start"] = df_values.periode.str[4:8]
    df_values["periode_end"] = df_values.periode.str[-4:]
    df_values["is_kabisat"] = 0
    
    df_values = df_values.merge(df_bridging_citra, left_on=['periode_start','periode_end','is_kabisat'],  right_on=['periode_start','periode_end','is_kabisat'])
    df_values['year_id_per_image'] = df_values.periode.str[:4]+"_"+df_values.id_per_image.astype("str").str.zfill(2)
    
    df_VH_wide_res = df_values.sort_values('year_id_per_image').pivot(index='idpoint', columns='year_id_per_image', values=band).reset_index()
    df_VH_wide_res = pd.concat([df_all_wide, df_VH_wide_res], axis=0)
    df_VH_wide_res['idsubsegmen'] = df_VH_wide_res.idpoint.str[:-3]
    return df_VH_wide_res

In [10]:
df_repair = pd.read_csv("/data/raw/processed/data_ksa_long_relabeled_repair.csv")
df_repair.head(2)

,Unnamed: 0,idsegmen,idsubsegmen,tahun,bulan,obs,idsubsegmen_repair,idsubsegmen_old,nth,id_x,observation,class,idprov
0,0,110101001,A1,22,1,8.0,110101001A1,110101001A1,0,110101001A1,8.0,NV,11
1,1,110101001,A2,22,1,4.0,110101001A2,110101001B1,0,110101001A2,4.0,H,11


In [11]:
df_repair["idkab"] = [x[:4] for x in df_repair.idsubsegmen_repair]
df_repair.loc[df_repair.idprov >= 90,"idkab"].unique()

array(['9101', '9104', '9105', '9106', '9107', '9108', '9111', '9401',
       '9402', '9403', '9404', '9412', '9413', '9414', '9417', '9419',
       '9420', '9426', '9471'], dtype=object)

In [27]:
df_repair

,Unnamed: 0,idsegmen,idsubsegmen,tahun,bulan,obs,idsubsegmen_repair,idsubsegmen_old,nth,id_x,observation,class,idprov,idkab
0,179262,910108319,A1,22,1,1.00,910108319A1,910108319A1,0,910108319A1,1.0,V1,91,9101
1,179263,910108319,A2,22,1,1.00,910108319A2,910108319B1,0,910108319A2,1.0,V1,91,9101
2,179264,910108319,A3,22,1,1.00,910108319A3,910108319C1,0,910108319A3,1.0,V1,91,9101
3,179265,910108319,B1,22,1,1.00,910108319B1,910108319A2,0,910108319B1,1.0,V1,91,9101
4,179266,910108319,B2,22,1,1.00,910108319B2,910108319B2,0,910108319B2,1.0,V1,91,9101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20515,4322803,947101006,B2,23,12,7.99,947101006B2,947101006B2,23,947101006B2,7.0,NP,94,9471
20516,4322804,947101006,B3,23,12,7.70,947101006B3,947101006C2,23,947101006B3,7.0,NP,94,9471
20517,4322805,947101006,C1,23,12,7.30,947101006C1,947101006A3,23,947101006C1,7.0,NP,94,9471
20518,4322806,947101006,C2,23,12,7.99,947101006C2,947101006B3,23,947101006C2,7.0,NP,94,9471


In [14]:
# bridng_papua

In [28]:
bridng_papua=pd.read_excel('/data/ksa/00_Data_Input/bridging_papua.xlsx')
bridng_papua['KAB2022_2']=bridng_papua.KAB2022_2.astype(str)
bridng_papua['KAB2023_1']=bridng_papua.KAB2023_1.astype(str)
bridng_papua.head(2)

,SEMESTER,KAB2022_2,NMKB2022_2,KAB2023_1,NMKB2023_1,ID_PERUB,KET_PERUB,KD_DASHUK,DASHUK
0,SEMESTER 1 2023,9101,FAKFAK,9101,FAKFAK,3,GANTI_KODE,JENIS_UNDANG_UNDANG,UNDANG UNDANG NO. 29 TAHUN 2022
1,SEMESTER 1 2023,9102,KAIMANA,9102,KAIMANA,3,GANTI_KODE,JENIS_UNDANG_UNDANG,UNDANG UNDANG NO. 29 TAHUN 2022


In [16]:
df_repair['idkab']=df_repair.idsubsegmen_old.apply(lambda y: y[:4])
df_repair_khusus=df_repair.merge(bridng_papua,left_on='idkab',right_on='KAB2022_2',how='inner')
df_repair_khusus['idsubsegmen_repair']=df_repair_khusus.apply(lambda y: y.KAB2023_1+y.idsubsegmen_repair[4:],axis=1)
df_repair_khusus['idsubsegmen_old']=df_repair_khusus.apply(lambda y: y.KAB2023_1+y.idsubsegmen_old[4:],axis=1)
df_repair=df_repair_khusus[df_repair.columns]
df_repair.head(2)

,Unnamed: 0,idsegmen,idsubsegmen,tahun,bulan,obs,idsubsegmen_repair,idsubsegmen_old,nth,id_x,observation,class,idprov,idkab
0,179262,910108319,A1,22,1,1.0,910108319A1,910108319A1,0,910108319A1,1.0,V1,91,9101
1,179263,910108319,A2,22,1,1.0,910108319A2,910108319B1,0,910108319A2,1.0,V1,91,9101


In [21]:
df_repair.idkab.unique()

array(['9101', '9104', '9105', '9106', '9107', '9108', '9111', '9401',
       '9402', '9403', '9404', '9412', '9413', '9414', '9417', '9419',
       '9420', '9426', '9471'], dtype=object)

In [22]:
## Cek error mgrs
mgrs_done_ = os.listdir('/data/ksa/03_Sampling/data/'+idprov+'/') 
mgrs_done_ = [x for x in mgrs_done_ if ".pkl" in x]
mgrs_done_ = [x[-9:-4] for x in mgrs_done_]
mgrs_done_

['54LVR', '54LVS', '54MUT', '54MVT']

In [24]:
# replacement_rules = {
#     'A1': 'A1',
#     'A2': 'B1',
#     'A3': 'C1',
#     'B1': 'A2',
#     'B2': 'B2',
#     'B3': 'C2',
#     'C1': 'A3',
#     'C2': 'B3',
#     'C3': 'C3',
# }
# def replace_strings(input_string, rules):
#     words = input_string.split('#')
#     words1=words[0][:-2]+''+rules[words[0][-2:]]
#     return words1

In [25]:
for mgrs in tqdm(mgrs_done_):
    df_values = get_df_values(data, idprov, mgrs)
    break

  0%|          | 0/4 [00:00<?, ?it/s]


In [26]:
df_values

,periode,idpoint,MGRS,weight,Sigma0_VH_db_interpolate,VH
0,20210101_20210112,950103017C2#01,54LVR,1,-19.067594,-18.419228
1,20210113_20210124,950103017C2#01,54LVR,1,-18.471379,-19.762608
2,20210125_20210205,950103017C2#01,54LVR,0,-17.745491,-17.745491
3,20210206_20210217,950103017C2#01,54LVR,1,-17.018502,-16.726540
4,20210218_20210301,950103017C2#01,54LVR,0,-16.418984,-16.418984
...,...,...,...,...,...,...
951745,20231028_20231108,950104003C1#07,54LVR,0,-17.334649,-17.334649
951746,20231109_20231120,950104003C1#07,54LVR,1,-17.247538,-16.389412
951747,20231121_20231202,950104003C1#07,54LVR,0,-17.278356,-17.278356
951748,20231203_20231214,950104003C1#07,54LVR,1,-17.368139,-17.662962


In [21]:
for mgrs in tqdm(mgrs_done_):
    df_values = get_df_values(data, idprov, mgrs)
    df_VH_wide = reformat_to_wide(df_values, "VH", df_bridging_citra, df_all_wide)
    df_VH_wide['idsubsegmen_repair']=df_VH_wide.idpoint.apply(lambda y:
                                                              replace_strings(y,replacement_rules))
    df_full = df_VH_wide.copy()
    
    #df_full = df_full.merge(df_repair[['idsubsegmen_repair','idsubsegmen_old','tahun', 'bulan', 
    #                                   'observation', 'class', 'year_id_per_image']], how="left", left_on = "idsubsegmen", right_on="idsubsegmen_old")
    print(df_full.head(2))
    
    df_full['kdpoint'] = df_full['idpoint'].str[-3:]
    df_full['idpoint'] = df_full['idsubsegmen_repair'] +  df_full['kdpoint']
    df_full['idsubsegmen'] = df_full['idsubsegmen_repair']
    year_id_per_image_=['2022_03', '2022_05', '2022_08', '2022_10', '2022_13', '2022_15',
       '2022_18', '2022_20', '2022_23', '2022_25', '2022_28', '2022_30',
       '2023_03', '2023_05', '2023_08', '2023_10', '2023_13', '2023_15',
       '2023_18', '2023_20', '2023_23', '2023_25', '2023_28', '2023_30']
    df_wide_full = pd.DataFrame()
    for yi in year_id_per_image_:
        df_tmp = df_full.loc[df_full.year_id_per_image == yi]
        if df_tmp.shape[0]>1:
            #print(yi)
            df_tmp.loc[:,'MGRS'] = mgrs
            ind = df_VH_wide.columns.to_list().index(yi)+1
            df_wide_tmp = pd.concat([df_VH_wide.iloc[:,0:1], df_VH_wide.iloc[:,ind-31:ind]], axis=1)
            df_wide_res = df_tmp.merge(df_wide_tmp, how='left', left_on='idpoint', right_on='idpoint')
            df_wide_res.columns.values[-31:] = vh_list
            df_wide_full = pd.concat([df_wide_full,df_wide_res], axis=0) 
        # break
    # break
    # if(data == 'not imputed'):
    #     with open('/data/ksa/03_Sampling/data-wide/'+idprov+'/wide_data_'+mgrs+'.pkl', 'wb') as f:
    #         pickle.dump(df_wide_full, f)
    # elif(data == 'imputed'):
    #     with open('/data/ksa/04_Data_Preprocessing/'+idprov+'/01_imputation/wide_data/wide_data_'+mgrs+'.pkl', 'wb') as f:
    #         pickle.dump(df_wide_full, f)
    df_wide_full
    break

  0%|          | 0/4 [00:05<?, ?it/s]

          idpoint    2021_01    2021_02    2021_03    2021_04    2021_05  \
0  950103009A1#01 -22.559929 -22.004097 -20.120438 -18.878666 -17.647390   
1  950103009A1#02 -19.612625 -19.583176 -17.903352 -16.664980 -16.222032   

     2021_06    2021_07    2021_08    2021_09  ...    2023_23    2023_24  \
0 -16.110752 -16.166317 -15.506385 -15.771246  ... -21.650313 -22.650072   
1 -15.016832 -15.948313 -15.110939 -14.758004  ... -18.690449 -18.523705   

     2023_25    2023_26    2023_27    2023_28    2023_29    2023_30  \
0 -23.050606 -22.957619 -23.060585 -22.819234 -22.253119 -22.396893   
1 -21.364384 -20.088282 -19.922634 -19.984058 -19.510117 -19.638789   

   idsubsegmen  idsubsegmen_repair  
0  950103009A1         950103009A1  
1  950103009A1         950103009A1  

[2 rows x 93 columns]


AttributeError: 'DataFrame' object has no attribute 'year_id_per_image'

In [ ]:
df_wide_full

## samplign_VH / sampling_VV

In [ ]:
# recode_dict = {
#     'V1': '1.0',
#     'V2': '2.0',
#     'G': '3.0',
#     'H': '4.0',
#     'PL': '5.0',
#     'P': '99.0',
#     'NP': '6.0',
#     'NV': '7.0',
#     'BL': '0.0'
# }

# mgrs_ = ['48MXT', '48MYT', '48MXU', '48MXS', '48MYS', '48MZT', '48MZS',
#        '49MAN', '49MAM', '49MBM', '49MBN', '49MAP', '49MBP', '48MZU',
#        '48MYU']
# mgrs

recode_dict = {
    'V1':1,
    'V2':2,
    'G':3,
    'H':4,
    'BP':0,
    'PL':5,
    'BPL':0,
    'NP':6,
    'NV':6,
    None:99
}

In [ ]:
#df_sampling['class'].unique()

In [ ]:
for mgrs in tqdm(mgrs_done_):
    if(data == 'not imputed'):
        with open('/data/ksa/03_Sampling/data-wide/'+idprov+'/wide_data_'+mgrs+'.pkl', 'rb') as f:
            df_sampling = pickle.load(f)
    elif(data == 'imputed'):
        with open('/data/ksa/04_Data_Preprocessing/'+idprov+'/01_imputation/wide_data/wide_data_'+mgrs+'.pkl', 'rb') as f:
            df_sampling = pickle.load(f)

    df_sampling['observation'] = df_sampling['class'].replace(recode_dict)
    df_sampling = df_sampling.loc[df_sampling.observation != 'P']
    # df_sampling = df_sampling.loc[df_sampling.observation != '99.0']
    df_sampling = df_sampling.loc[df_sampling.observation != 99]
    df_sampling['idsegment'] = df_sampling['idsubsegmen'].str[:-2]
    df_sampling.rename(columns={'idsubsegmen': 'idsubsegment'}, inplace=True)
    df_sampling.rename(columns={'bulan': 'nth'}, inplace=True)
    df_sampling.rename(columns={'year_id_per_image': 'periode'}, inplace=True)
    
    df_sampling = df_sampling[['idpoint','idsubsegment','idsegment','nth','periode',
     'observation','class', 'MGRS', 'VH_30', 'VH_29', 'VH_28', 'VH_27', 'VH_26', 'VH_25',
     'VH_24', 'VH_23', 'VH_22', 'VH_21', 'VH_20', 'VH_19', 'VH_18', 'VH_17',
     'VH_16', 'VH_15', 'VH_14', 'VH_13', 'VH_12', 'VH_11', 'VH_10', 'VH_9',
     'VH_8', 'VH_7', 'VH_6', 'VH_5', 'VH_4', 'VH_3', 'VH_2', 'VH_1', 'VH_0']]

    # df_sampling = df_sampling[['idpoint','idsubsegment','idsegment','nth','periode',
    #  'observation', 'class','MGRS', 'VV_30', 'VV_29', 'VV_28', 'VV_27', 'VV_26', 'VV_25',
    #  'VV_24', 'VV_23', 'VV_22', 'VV_21', 'VV_20', 'VV_19', 'VV_18', 'VV_17',
    #  'VV_16', 'VV_15', 'VV_14', 'VV_13', 'VV_12', 'VV_11', 'VV_10', 'VV_9',
    #  'VV_8', 'VV_7', 'VV_6', 'VV_5', 'VV_4', 'VV_3', 'VV_2', 'VV_1', 'VV_0']]
    
    if(data == 'not imputed'):
        with open('/data/ksa/03_Sampling/data-wide/'+idprov+'/sampling_VH_'+mgrs+'.pkl', 'wb') as f:
                pickle.dump(df_sampling, f)
    elif(data == 'imputed'):
        with open('/data/ksa/04_Data_Preprocessing/'+idprov+'/01_imputation/wide_data/sampling_VH_'+mgrs+'.pkl', 'wb') as f:
            pickle.dump(df_sampling, f)

    # if(data == 'not imputed'):
    #     with open('/data/ksa/03_Sampling/data-wide/32/sampling_VV_'+mgrs+'.pkl', 'wb') as f:
    #             pickle.dump(df_sampling, f)
    # elif(data == 'imputed'):
    #     with open('/data/ksa/04_Data_Preprocessing/32/01_imputation/wide_data/sampling_VV_'+mgrs+'.pkl', 'wb') as f:
    #         pickle.dump(df_sampling, f)
            
    
    # break


In [ ]:
df_sampling.observation.unique()

In [ ]:
df_sampling.groupby(['observation','class']).size()